# Data Pre-processing & Feature Selection

In [ ]:
###########################################################################
#
#  Copyright 2021 Google Inc.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This solution, including any related sample code or data, is made available 
# on an “as is,” “as available,” and “with all faults” basis, solely for 
# illustrative purposes, and without warranty or representation of any kind. 
# This solution is experimental, unsupported and provided solely for your 
# convenience. Your use of it is subject to your agreements with Google, as 
# applicable, and may constitute a beta feature as defined under those 
# agreements.  To the extent that you make any data available to Google in 
# connection with your use of the solution, you represent and warrant that you 
# have all necessary and appropriate rights, consents and permissions to permit 
# Google to use and process that data.  By using any portion of this solution, 
# you acknowledge, assume and accept all risks, known and unknown, associated 
# with its usage, including with respect to your deployment of any portion of 
# this solution in your systems, or usage in connection with your business, 
# if at all.
###########################################################################

## 0) Dependencies

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = '' #add proj name

In [ ]:
# Google credentials authentication libraries
from google.colab import auth
auth.authenticate_user()

import sys

# data processing libraries
import numpy as np
from numpy.core.numeric import NaN
import datetime
import pandas as pd
import pandas_gbq

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
!pip install boruta #boruta for feature selection
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier

# modeling and metrics
from scipy.optimize import least_squares
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm


import itertools
from scipy.stats.stats import pearsonr

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5] #change size of plot
import seaborn as sns
import plotly.express as px

# BigQuery Magics
'''
BigQuery magics are used to run BigQuery SQL queries in a python environment.
These queries can also be run in the BigQuery UI
'''

from google.cloud import bigquery
from google.cloud.bigquery import magics

magics.context.project = project_name  #update your project name 

client = bigquery.Client(project=magics.context.project)

## 1) Import dataset

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

In [ ]:
%%bigquery df
SELECT *
FROM `.RBA_demo.SAMPLE_DATA`; #update with project name

In [ ]:
df.head()

The first step is to remove variables that won't be used in the model. In this example, we remove columns like geo which is consistent across the dataset and aggregated media such as total clicks across DSPs.


In [ ]:
df.drop(columns = ['geo','x1','x2','x8','x18','x19','x20','x21','x22','x23','x24','x25'], inplace = True)

In [ ]:
len(df.columns)

In [ ]:
df.describe()

In [ ]:
#Set the date as index
date_col = "date" #@param {type:"string"}
df = df.sort_values(date_col).set_index(date_col)

Option to aggregate daily data to weekly data

In [ ]:
is_daily_data = False #@param {type:"boolean"}
#if you are using weekly data, make sure this is False. If using daily data, set to True.

In [ ]:
if is_daily_data == False:
  df = df.resample('7D').sum() #aggregate daily data to weekly

In [ ]:
df.head()

## 2) Data Cleaning

### 2.1) Check for missing data and impute

Check the amount of of missing values (% of total column) in the data and sort by 
highest to lowest.

In [ ]:
missing_values = 100*df.isnull().sum()/len(df)
missing_values.sort_values(ascending = False)

If there are any NAs in the data that should be zeros, replace those data
points with zero.

In [ ]:
df.fillna(0, inplace = True)

## 3) Define Y (KPI column) and create initial feature set

In [ ]:
#Input column names for Y (ex: "new_accounts" or "sales") 
kpi_col = "y1" #@param {type:"string"}
target_variable = df[kpi_col] #y variable

In [ ]:
# Create a dataframe for features (all variables except date and kpi) x variables
featureset_df = df[df.columns[df.columns != date_col]]
featureset_df = df[df.columns[df.columns != kpi_col]]

In [ ]:
featureset_df.head()

## 4) Visualize Series

Optional:

Visualizing each series is useful to better understand the underlying distribution of the data. This allows for examination of outliers. 

Understanding the distribution of the underlying data can also inform prior parameterization in bayesian modeling approaches later on.

In [ ]:
for i in range(2,len(featureset_df.columns)):
  plt.figure()
  sns.kdeplot(featureset_df[featureset_df.columns[i]], label = featureset_df.columns[i], shade = True)

## 5) Feature Creation

### 5.1) Check for Seasonality and add Flag

View the target variable as a time series plot and identify periods where data peaks.

We also add flags for periods of peak seasonality such as Q2, Q3, and major winter holidays.


In [ ]:
fig = px.line(df[kpi_col])
fig.show()

In [ ]:
featureset_df['Is_Q2Q3'] = (df.index.get_level_values(0).month == 4).astype(int) | (df.index.get_level_values(0).month == 5).astype(int) | (df.index.get_level_values(0).month == 6).astype(int) | (df.index.get_level_values(0).month == 7).astype(int) | (df.index.get_level_values(0).month == 8).astype(int) | (df.index.get_level_values(0).month == 9).astype(int)
featureset_df['Is_Holiday'] = ((df.index == '2017-11-17') | (df.index == '2017-12-22') | (df.index == '2018-11-16') | (df.index == '2018-12-21') | (df.index == '') | (df.index == ''))

### 5.2) Lag, Diminishing Returns, Adstock

We'll need to transform the raw data by applying lag, diminishing returns, and adstock returns to have it most accurately predict the target variable. 


- We define lag as the impact of media on sales "n" days after it was served.

- We define diminishing returns as the saturation point media will hit after a certain amount of spend, thereby becoming less effective for each additional dollar spent

- We define adstock as the effect of media spend across a number of days




First, split the df into two different dataframes:

1. Features that don't need to be transformed
      - Examples are: 
          - date
          - target variable
          - control variables (seasonality, promotions, etc.)

2. Features that do need to be transformed
      -  Paid media tactics 
      -  Any other feature where there is some sort of delayed response with the target variable


Starting points for lags:
- If you are using daily data, the lag should at default be 14.
- If you are using weekly data, the lag should at default be 5.

Others can and should be tested to determine the best lag length for your specific data.


In [ ]:
# Variables that do not need to be transformed

untransformed_df = pd.concat([target_variable, featureset_df[['Is_Q2Q3','Is_Holiday']]], axis = 1) #Target variable + controls

In [ ]:
# Variables that do need to be transformed

#exclude dummies/controls that do not need to be transformed
#transformed_df = featureset_df[['feature1', 'feature2',...]]

'''
Note: In this example case, almost all of the features in the featureset_df are media features.
As more dummy variables or other control variables are added, the user will need to 
specify which columns should be transformed
'''
transformed_df = featureset_df.loc[:,~featureset_df.columns.isin(['Is_Q2Q3','Is_Holiday'])]

#### 5.2.1) Create the transformation functions

In [ ]:
# This function builds the values for the diminishing returns curve, which are
# later applied in the transformation step

def buildDReturnsValues(index, original_column, percent):
  if index == 0:
    return [original_column[0] * percent]
  else:
    previous_values = buildDReturnsValues(index-1, original_column, percent)
    previous_values.append(original_column[index] * percent + previous_values[index-1] * (1-percent))
    return previous_values

In [ ]:
#This step can take several minutes
#This creates all combinations and then calculates the correlation between each variable and the Y variable. 
#Returns the top 3 highest correlated features

def create_transformations(df1, df2):
  columns = df2.columns
  sales = df1[[kpi_col]]
  all_data = [] 
  for col in columns:
    newdf = Transformation(df2, col, True)
    corr_df = pd.concat([sales, newdf], axis=1)
    corr = abs(corr_df.corr().sort_values(kpi_col, ascending=False))
    new_vals= corr.iloc[1:4 , 0:1].index.tolist()
    data = newdf[new_vals]
    all_data.append(data)
  final_data = pd.concat(all_data,axis=1)
  return final_data

In [ ]:
#This function creates every combination of Lag, Adstock (carryover), and Diminishing Returns Shape
#The only thing that needs to be updated is this function is the lag  
# If you are using weekly data, keep range(0, 4, 1). This is testing lags from 0 - 4 weeks
# If you are using daily data, sugestion to update  range(0, 4, 1) to  range(0, 14, 1). This will test lags from 0 - 14 days.

def Transformation(dataframe, x, is_daily_data = is_daily_data):
    lag = []
    for i in range(0, 14 if is_daily_data else 4 , 1):
        data = dataframe[x].shift(i).to_frame()
        data.columns = [col_name+'lag'+str(i)for col_name in data.columns]
        lag.append(data)
    lag = pd.concat(lag,axis=1)
    lag = lag.fillna(0)
    dreturns = []
    for i in np.linspace(0.6,1.0,num=5):
      data = pow(lag,i)
      data.columns = [col_name+'dreturns'+str(i)for col_name in data.columns]
      dreturns.append(data)
    dreturns = pd.concat(dreturns,axis=1) 
    adstock=[]
    #j = 0
    for percent in np.linspace(0.6,1.0,5):
        data = dreturns.copy()
        data.columns = [col_name+'adstock'+str(percent)for col_name in data.columns]
        for j in range(0, len(dreturns.columns)):
          data[data.columns[j]] = buildDReturnsValues(len(data[data.columns[j]])-1, data[data.columns[j]], percent)
        
        adstock.append(data)
        #j = j + 1
    adstock = pd.concat(adstock,axis=1)
    
    return adstock

#### 5.2.1) Implement the transformations

Make sure data is correctly sorted by date before running feature selection algo.

This is important because the algorithm takes from a previous row of the data as it evaluates the current row. Unsorted data can cause errors in resulting feature selection
info.


In [ ]:
transformed_df = transformed_df.sort_values(date_col)
sys.setrecursionlimit(len(transformed_df.index)+100)

transformed_df = create_transformations(untransformed_df, transformed_df)
transformed_df.head()

## 6) Feature Selection

For feature selection we employ the Boruta algorithm.[(More information here)](https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a
)

This algorithm will tell you the rank of each feature and whether or not to keep a varaible in the model (i.e. Keep  = True/False). The goal of RBA is to optimize across all paid digital media tactics, therefore select the top ranking feature for each group of features (whether or not the algorithm tells you to keep the feature).


In [ ]:
# Specifiying the target and x variables
y = target_variable
x = transformed_df #update with transformed features

In [ ]:
# define random forest classifier
forest = RandomForestRegressor(n_jobs=-1, max_depth=5)
forest.fit(x, y)

In [ ]:
# define Boruta feature selection method
feat_selector = BorutaPy(forest, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features
feat_selector.fit(np.array(x), np.array(y))

# check selected features
feat_selector.support_

# check ranking of features
feat_selector.ranking_

# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(np.array(x))

In [ ]:
#Select the top ranking variable for each group of variables. 
feature_ranks = list(zip(x.columns, 
                         feat_selector.ranking_, 
                         feat_selector.support_))

# iterate through and print out the results
for feat in feature_ranks:
    print('{:<25}, Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))

Reduce the overall dataset to just selected features using the ranking
from the Boruta output, and save to a dataframe

In [ ]:
selected_featureset_df = transformed_df[['x3lag12dreturns0.8adstock1.0',
'x4lag0dreturns1.0adstock1.0',
'x5lag11dreturns0.9adstock0.6',
'x6lag0dreturns1.0adstock1.0',
'x7lag12dreturns1.0adstock0.9',
'x9lag0dreturns0.7adstock1.0',
'x10lag0dreturns0.9adstock1.0',
'x11lag11dreturns1.0adstock0.7',
'x12lag0dreturns0.9adstock1.0',
'x13lag12dreturns0.9adstock1.0',
'x14lag0dreturns1.0adstock1.0',
'x15lag12dreturns0.9adstock1.0',
'x16lag12dreturns0.6adstock0.7',
'x17lag0dreturns0.7adstock1.0',
'x26lag12dreturns0.6adstock0.6',
'x27lag0dreturns1.0adstock0.9',
'x28lag9dreturns0.8adstock0.6',
'x29lag12dreturns0.6adstock0.8',
'x30lag0dreturns1.0adstock1.0',
'x31lag0dreturns1.0adstock0.8',
'x32lag0dreturns0.6adstock0.6',
'x33lag12dreturns0.6adstock0.6',
'x34lag13dreturns1.0adstock0.6',
'x35lag12dreturns0.6adstock0.9',
'x36lag8dreturns1.0adstock0.6',
'x37lag12dreturns0.6adstock0.8',
'x38lag0dreturns0.7adstock0.9',
'x39lag12dreturns1.0adstock0.8',
'x40lag12dreturns1.0adstock0.6',
'x41lag12dreturns0.6adstock1.0',
'x42lag3dreturns1.0adstock0.6',
'x43lag4dreturns0.6adstock0.9',
'x44lag11dreturns0.6adstock0.8',
'x45lag0dreturns0.6adstock0.7',
'x46lag0dreturns0.6adstock0.6'
]]

In [ ]:
# add back in the untransformed control variables to the featureset
selected_featureset_df = pd.concat(
    [selected_featureset_df,untransformed_df[untransformed_df.columns[untransformed_df.columns != kpi_col]]],
    axis = 1)

## 7) Feature Scaling

### 7.1) Feature Scaling

The default method of standardization utilizes Standard Scaler, which takes in
input data and transforms so that the output has mean 0 and standard deviation of 1
across all features.


Alternative methods of feature scaling include square-root transformation,
de-meaning, natural log transformations, Min-Max Scalers, or normalization

In [ ]:
scaler = StandardScaler()
standardized_transform = scaler.fit_transform(selected_featureset_df)
selected_featureset_df = pd.DataFrame(standardized_transform, columns = selected_featureset_df.columns)

Option to review visuals of the data. After the data is standardized the distributions may take on a more normal shape.

In [ ]:
'''
for i in range(0,len(X_transform_stand.columns)):
  plt.figure()
  sns.kdeplot(X_transform_stand[X_transform_stand.columns[i]], label = X_transform_stand.columns[i], shade = True)
'''

In [ ]:
selected_featureset_df.head()

## 8) Handle Multicollinearity (reduce feature set)

1. Print a correlation heatmap to visualize correlations across feature set
2. Run variance inflation factor analysis and output results to flag multicollinearity above specified threshold

In [ ]:
correl = selected_featureset_df.corr()

# Getting the Upper Triangle of the co-relation matrix
matrix = np.triu(correl)

# using the upper triangle matrix as mask 
sns.heatmap(correl, mask=matrix)

Run VIF analysis and flag values greater than 10.

Industry best practice flags values above 10 as an extreme violation of regression model assumptions. [(Reference)](https://en.wikipedia.org/wiki/Variance_inflation_factor)


In [ ]:
vif = add_constant(selected_featureset_df)

# loop to calculate the VIF for each X 
vif = pd.Series([variance_inflation_factor(vif.values, i) 
      for i in range(vif.shape[1])], 
      index=vif.columns) 

In [ ]:
# processing to output VIF results as a dataframe 
vif_df=vif.to_frame().reset_index()

vif_df.columns = ['feature', 'vif']
vif_df=vif_df.replace([np.inf], np.nan) # replace inf calculations as missing and zero fill 
vif_df=vif_df.fillna(0).sort_values(by="vif", ascending=False)

In [ ]:
vif_df.reset_index(inplace = True)
vif_df

Drop the highest VIF features and print the high collinearity columns in a list

In [ ]:
high_collinearity_columns = vif_df.feature[vif_df['vif'] >= 10].to_list()
high_collinearity_columns

Drop 1 variable at a time (start with the highest VIF) and re-run the VIF cell to re-check multicollinearity. This will allow the user to preserve as many features in the model as possible.

In [ ]:
cols_to_drop = []
while vif_df.vif[1] >= 10:
  if vif_df.vif[1] >= 10:
    cols_to_drop.append(vif_df.feature[1])
    selected_featureset_df.drop(columns = vif_df.feature[1],inplace = True) 
    vif = add_constant(selected_featureset_df)
  # loop to calculate the VIF for each X 
    vif = pd.Series([variance_inflation_factor(vif.values, i) 
    for i in range(vif.shape[1])], index=vif.columns) 
    # processing to output VIF results as a dataframe 
    vif_df=vif.to_frame().reset_index()
    vif_df.columns = ['feature', 'vif']
    vif_df=vif_df.replace([np.inf], np.nan) # replace inf calculations as missing and zero fill 
    vif_df=vif_df.fillna(0).sort_values(by="vif", ascending=False)
    vif_df.reset_index(inplace = True)

In [ ]:
cols_to_drop

In [ ]:
selected_featureset_df.columns

In [ ]:
len(selected_featureset_df.columns)

In [ ]:
# Replace the decimal points with underscores so that data can be exported to BQ
selected_featureset_df.columns = selected_featureset_df.columns.str.replace(".","_")

## 9) Export Final Dataset

In [ ]:
df[df.columns[df.columns.isin(parse_final_column_features(selected_featureset_df.columns))]]

In [ ]:
untransformed_df[untransformed_df.columns[untransformed_df.columns.isin(parse_final_column_features(selected_featureset_df.columns))]]

### 9.1) Trim the final dataset according to lag

In [ ]:
final_df = selected_featureset_df
final_df[kpi_col] = target_variable.reset_index()[kpi_col]

In [ ]:
final_df[date_col] = df.index #add back in the date as a separate column from the index

Trim the start of your dataset to correspond with the max lag
(for example: if max lag is 4 weeks, trim the first 4 weeks off of the data)

In [ ]:
max_lag = 13
final_df = final_df[max_lag:]
final_df.reset_index(inplace = True)
final_df.drop(columns = 'index',inplace = True)

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

In [ ]:
destination_project_id = "" #@param
destination_dataset = "RBA_demo" #@param
destination_table = "cleaned_data" #@param
dataset_table = destination_dataset+"."+destination_table

final_df.to_gbq(dataset_table, 
                 destination_project_id,
                 chunksize=None, 
                 if_exists='replace'
                 )

### 9.2) Prepare the data for optimization tool

The budget optimization tool requires the model features in their original un-transformed state as an input. The following function pulls the names of the required columns and collects the data from the relevant dataframes.

In [ ]:
def parse_final_column_features(columns):
  final = []
  for col in columns:
    splitList = col[::-1].split('gal', 1)
    parsed = (splitList[1] if len(splitList) > 1 else splitList[0])
    final.append(parsed[::-1])
  return final

In [ ]:
optimizer_df = df[df.columns[df.columns.isin(parse_final_column_features(selected_featureset_df.columns))]]
optimizer_df = optimizer_df.merge(untransformed_df[untransformed_df.columns[untransformed_df.columns.isin(parse_final_column_features(selected_featureset_df.columns)) & (untransformed_df.columns != kpi_col)]],how = 'inner',on = ['date'])

In [ ]:
destination_project_id = "" #@param
destination_dataset = "RBA_demo" #@param
destination_table = "optimizer_data" #@param
dataset_table = destination_dataset+"."+destination_table

optimizer_df.to_gbq(dataset_table, 
                 destination_project_id,
                 chunksize=None, 
                 if_exists='replace'
                 )